#`6431309721 Chanagun Viriyasathapornpong`

In [755]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

from subprocess import check_output

from sklearn.metrics import confusion_matrix, classification_report

#1) Load DataSet

In [756]:
!rm mushroom2020_dataset.csv
!wget https://raw.githubusercontent.com/pvateekul/2110446_DSDE_2023s2/main/code/Week03_ML/mushroom2020_dataset.csv

--2024-01-29 14:08:54--  https://raw.githubusercontent.com/pvateekul/2110446_DSDE_2023s2/main/code/Week03_ML/mushroom2020_dataset.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358413 (350K) [text/plain]
Saving to: ‘mushroom2020_dataset.csv’

mushroom2020_datase 100%[===================>] 350.01K  --.-KB/s    in 0.1s    

2024-01-29 14:08:54 (3.14 MB/s) - ‘mushroom2020_dataset.csv’ saved [358413/358413]



In [757]:
df = pd.read_csv('mushroom2020_dataset.csv') # load data set

In [758]:
df.head() # first 5 row

,id,label,cap-shape,cap-surface,bruises,odor,gill-attachment,gill-spacing,gill-size,stalk-shape,...,ring-number,ring-type,spore-print-color,population,habitat,cap-color-rate,gill-color-rate,veil-color-rate,stalk-color-above-ring-rate,stalk-color-below-ring-rate
0,1,p,x,s,t,p,f,c,n,e,...,o,p,k,s,u,1.0,3.0,1.0,1.0,1.0
1,2,e,x,s,t,a,f,c,b,e,...,o,p,n,n,g,2.0,3.0,1.0,1.0,1.0
2,3,e,b,s,t,l,f,c,b,e,...,o,p,n,n,m,3.0,1.0,1.0,1.0,1.0
3,4,p,x,y,t,p,f,c,n,e,...,o,p,k,s,u,3.0,1.0,1.0,1.0,1.0
4,5,e,x,s,f,n,f,w,b,t,...,o,e,n,a,g,4.0,3.0,1.0,1.0,1.0


#2) Drop rows where the target (label) variable is missing.

In [759]:
df["gill-size"].isnull().sum() # numbrt of "na" in "gill-size" columns

121

In [760]:
len(df) # number of row

5824

In [761]:
df.dropna(subset=["label"], inplace=True) # drop missing row
len(df)

5764

In [762]:
df.label.isnull().sum() # check that label varialble dont have missing variablefor sure

0



# 3) Drop the following variables:







'id','gill-attachment', 'gill-spacing', 'gill-size','gill-color-rate', 'stalk-root', 'stalk-surface-above-ring', 'stalk-surface-below-ring', 'stalk-color-above-ring-rate','stalk-color-below-ring-rate','veil-color-rate','veil-type'

In [763]:
df.shape[1] # number of column

24

In [764]:
drop_cols = ['id','gill-attachment', 'gill-spacing', 'gill-size', # column that need to drop
             'gill-color-rate', 'stalk-root', 'stalk-surface-above-ring',
             'stalk-surface-below-ring', 'stalk-color-above-ring-rate',
             'stalk-color-below-ring-rate','veil-color-rate','veil-type']
drop_df = df.drop(columns=drop_cols) # drop column

In [765]:
drop_df.shape[1] # number column that already drop

12

In [766]:
drop_df.head() # first 5 row of dropped column dataset

,label,cap-shape,cap-surface,bruises,odor,stalk-shape,ring-number,ring-type,spore-print-color,population,habitat,cap-color-rate
0,p,x,s,t,p,e,o,p,k,s,u,1.0
1,e,x,s,t,a,e,o,p,n,n,g,2.0
2,e,b,s,t,l,e,o,p,n,n,m,3.0
3,p,x,y,t,p,e,o,p,k,s,u,3.0
4,e,x,s,f,n,t,o,e,n,a,g,4.0


#4) Examine the number of rows, the number of digits, and whether any are missing.

In [767]:
len(drop_df) # number of rows

5764

In [768]:
drop_df.shape[1] # number of digits

12

In [769]:
drop_df.isnull().sum() # show each number of missing digits

label                  0
cap-shape              0
cap-surface           27
bruises               99
odor                  99
stalk-shape          121
ring-number           62
ring-type             62
spore-print-color     56
population            56
habitat               31
cap-color-rate        27
dtype: int64

# 5) Fill missing values by adding the mean for numeric variables and the mode for nominal variables.

In [770]:
for col in drop_df.columns: #loop each column
  if drop_df[col].dtype == "object": # categorical
    drop_df[col].fillna(drop_df[col].mode()[0], inplace=True)
  else: # quantitative
      drop_df[col].fillna(drop_df[col].mean(), inplace=True)

In [771]:
drop_df.isnull().sum() # show each number of missing digits

label                0
cap-shape            0
cap-surface          0
bruises              0
odor                 0
stalk-shape          0
ring-number          0
ring-type            0
spore-print-color    0
population           0
habitat              0
cap-color-rate       0
dtype: int64

#6) Convert the label variable e (edible) to 1 and p (poisonous) to 0 and check the quantity. class0: class1

In [772]:
drop_df["label"] = drop_df["label"].apply(lambda x: 1 if x == "e" else 0)

In [773]:
drop_df.label.value_counts()

0    3660
1    2104
Name: label, dtype: int64

#7) Convert the nominal variable to numeric using a dummy code with drop_first = True.

In [774]:
nom_cols = []
for col in drop_df.columns:
  if drop_df[col].dtype == "object":
    nom_cols.append(col)

dummy_cols = pd.get_dummies(drop_df[nom_cols], drop_first=True)
df_dummy = pd.concat([drop_df, dummy_cols], axis=1)
df_dummy = df_dummy.drop(columns=nom_cols)

In [775]:
df_dummy

,label,cap-color-rate,cap-shape_c,cap-shape_f,cap-shape_k,cap-shape_x,cap-surface_g,cap-surface_s,cap-surface_y,bruises_t,...,population_n,population_s,population_v,population_y,habitat_g,habitat_l,habitat_m,habitat_p,habitat_u,habitat_w
0,0,1.0,0,0,0,1,0,1,0,1,...,0,1,0,0,0,0,0,0,1,0
1,1,2.0,0,0,0,1,0,1,0,1,...,1,0,0,0,1,0,0,0,0,0
2,1,3.0,0,0,0,0,0,1,0,1,...,1,0,0,0,0,0,1,0,0,0
3,0,3.0,0,0,0,1,0,0,1,1,...,0,1,0,0,0,0,0,0,1,0
4,1,4.0,0,0,0,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5819,1,1.0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
5820,1,1.0,0,0,0,1,0,1,0,0,...,0,0,1,0,0,1,0,0,0,0
5821,1,1.0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
5822,0,1.0,0,0,1,0,0,0,1,0,...,0,0,1,0,0,1,0,0,0,0


In [776]:
df_dummy.dtypes

label                    int64
cap-color-rate         float64
cap-shape_c              uint8
cap-shape_f              uint8
cap-shape_k              uint8
cap-shape_x              uint8
cap-surface_g            uint8
cap-surface_s            uint8
cap-surface_y            uint8
bruises_t                uint8
odor_c                   uint8
odor_f                   uint8
odor_l                   uint8
odor_m                   uint8
odor_n                   uint8
odor_p                   uint8
odor_s                   uint8
odor_y                   uint8
stalk-shape_t            uint8
ring-number_o            uint8
ring-number_t            uint8
ring-type_f              uint8
ring-type_l              uint8
ring-type_n              uint8
ring-type_p              uint8
spore-print-color_h      uint8
spore-print-color_k      uint8
spore-print-color_n      uint8
spore-print-color_o      uint8
spore-print-color_r      uint8
spore-print-color_w      uint8
spore-print-color_y      uint8
populati

#8) Split train/test with 20% test, stratify, and seed = 2020.

In [777]:
X = df_dummy.drop(columns=["label"])
y = df_dummy["label"]

In [778]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=2020)

In [779]:
len(X_train) # size of training set

4611

In [780]:
len(X_test) # size of test set

1153

#9) Create a Random Forest with GridSearch on training data with 5 CV.


- criterion':['gini','entropy']
- max_depth': [2,3,6]
- min_samples_leaf':[2,5,10]
- N_estimators':[100,200]
- random_state': 2020

In [ ]:
RFC = RandomForestClassifier()
RFC_params = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [2, 3, 6],
    'min_samples_leaf': [2, 5, 10],
    'n_estimators': [100, 200],
    'random_state': [2020],
}

RFC_rdm = GridSearchCV(estimator=RFC, param_grid=RFC_params, cv=5)
RFC_rdm.fit(X_train, y_train)

In [ ]:
RFC_rdm.best_params_

# 10) Predict the testing data set with confusion_matrix and classification_report.

In [ ]:
y_pred = RFC_rdm.predict(X_test) # predict using the trained model

In [ ]:
confusion_matrix(y_test, y_pred) # confusion_matrix

In [ ]:
classification_report(y_test, y_pred, digits=4) # classification_report